# Hello QCS

Welcome to QCS! Your QCS JupyterHub container is a virtual development and execution environment for quantum programming. It is hosted in close physical proximity to the QPUs in Rigetti’s quantum data center. This integrated platform eliminates the network latency found in web API access models, allowing you to iterate faster.For more information on this, check out our [documentation](https://docs.rigetti.com).

In this beginner's example, we are going to run a simple `hello_qcs()`. This function composes a simple quantum program, compiles it, and then runs it on the target device of your choice: either the QVM or a QPU. 

## Imports

To import `hello_qmi`, we must include in our path the folder in our QMI where it's located: one folder up from our current directory, or `..`.

In [1]:
import math
import os
import sys
from typing import Optional

from pyquil import Program, get_qc
from pyquil.api import QVM
from pyquil.gates import RX, MEASURE
from pyquil.quil import Pragma

## Run against the QVM Simulator

The function `hello_qmi` takes an optional argument naming the device to target (the QVM or a QPU). If no argument is provided, the default device is a `9q-generic-qvm`.

In [3]:
def hello_qcs(device_name: str = "9q-generic-qvm", shots: int = 5) -> None:
    """
    Get acquainted with your quantum computer by asking it to perform a simple
    coin-toss experiment. Involve 3 qubits in this experiment, and ask each one
    to give `shots` many results.

    :param device_name: The name of a quantum computer which can be retrieved
                        from `pyquil.api.get_qc()`. To find a list of all
                        devices, you can use `pyquil.api.list_devices()`.
    :param shots:       The number of times to run the Quil program steps
    """
    # Initialize your Quil program
    program = Program()

    # Allow the compiler to re-index to use the available topology
    program += Pragma('INITIAL_REWIRING', ['"PARTIAL"'])

    # Declare 3 bits of memory space for the readout results of all three qubits
    readout = program.declare('ro', 'BIT', 3)

    # For each qubit, apply a pulse to move the qubit's state halfway between
    # the 0 state and the 1 state
    program += RX(math.pi / 2, 0)
    program += RX(math.pi / 2, 1)
    program += RX(math.pi / 2, 2)

    # Add measurement instructions to measure the qubits and record the result
    # into the respective bit in the readout register
    program += MEASURE(0, readout[0])
    program += MEASURE(1, readout[1])
    program += MEASURE(2, readout[2])

    # This tells the program how many times to run the above sequence
    program.wrap_in_numshots_loop(shots)

    # Get the quantum computer we want to run our experiment on
    qc = get_qc(device_name)

    # Compile the program, specific to which quantum computer we are using
    compiled_program = qc.compile(program)

    # Run the program and get the shots x 3 array of results
    results = qc.run(compiled_program).readout_data.get('ro')
    # Print the results. We expect to see (shots x 3) random 0's and 1's
    print(f"Your{' virtual' if isinstance(qc.qam, QVM) else ''} quantum "
          f"computer, {device_name}, greets you with:\n", results)

In [4]:
hello_qcs()

Your virtual quantum computer, 9q-generic-qvm, greets you with:
 [[1 0 0]
 [0 1 0]
 [1 1 1]
 [0 0 0]
 [1 0 0]]


Congratulations! You just ran your first program against the QVM simulator.

## Run against the real QPU
To run against a real quantum computer, you only need to change the device argument to the `hello_qmi()` program. What QPU devices are available? We can use pyQuil to get a list.

In [5]:
# Query available devices (QPUs)
from pyquil.api._quantum_processors import list_quantum_processors

In [6]:
quantum_processor_names = list_quantum_processors()  # Available quantum processors are subject to change.
print(f"Available devices: {quantum_processor_names}.\n")

Available devices: ['Aspen-4', 'Aspen-7'].

Available lattices: ['Aspen-7-28Q-A', 'Aspen-7-25Q-B', 'Aspen-7-16Q-B', 'Aspen-7-15Q-B', 'Aspen-7-14Q-B', 'Aspen-7-13Q-B', 'Aspen-4-13Q-E', 'Aspen-7-12Q-B', 'Aspen-7-11Q-B', 'Aspen-7-10Q-B', 'Aspen-7-9Q-B', 'Aspen-4-9Q-E', 'Aspen-7-8Q-B', 'Aspen-4-8Q-E', 'Aspen-7-7Q-B', 'Aspen-4-7Q-E', 'Aspen-7-6Q-B', 'Aspen-4-6Q-E', 'Aspen-7-5Q-B', 'Aspen-4-5Q-E', 'Aspen-7-4Q-B', 'Aspen-7-4Q-C', 'Aspen-4-4Q-E', 'Aspen-4-4Q-D', 'Aspen-4-4Q-A', 'Aspen-4-3Q-A', 'Aspen-7-3Q-B', 'Aspen-4-3Q-E', 'Aspen-4-3Q-D', 'Aspen-4-2Q-C', 'Aspen-7-2Q-B', 'Aspen-4-2Q-A'].


## Where to go from here

This is just the beginning of your journey with QCS. Next, feel free to take a look under the hood at some of the QCS features that unlock the full potential of the platform for fast hybrid programming. These can be found in the other pre-loaded example notebooks; then, try writing some new notebooks of your own.

If you have any trouble getting things set up, please reach out to support@rigetti.com.

Happy quantum programming.

*-- The QCS Team*